In [ ]:
from agent_framework.tools.mcp_tool import MCPTool, MCPClient
from rich.pretty import pprint

client = MCPClient()
await client.connect_sse('http://localhost:8000/sse')
mcp_tools = await MCPTool.from_mcp_client(client)

{"timestamp": "2026-02-07T16:13:14.004906", "level": "INFO", "name": "httpx", "message": "HTTP Request: POST http://localhost:8000/messages/?session_id=be72cc977bdd485d98de699b36914f46 \"HTTP/1.1 202 Accepted\""}
{"timestamp": "2026-02-07T16:13:14.231389", "level": "INFO", "name": "httpx", "message": "HTTP Request: POST http://localhost:8000/messages/?session_id=be72cc977bdd485d98de699b36914f46 \"HTTP/1.1 202 Accepted\""}
{"timestamp": "2026-02-07T16:13:17.086188", "level": "INFO", "name": "httpx", "message": "HTTP Request: POST http://localhost:8000/messages/?session_id=be72cc977bdd485d98de699b36914f46 \"HTTP/1.1 202 Accepted\""}
{"timestamp": "2026-02-07T16:13:17.289978", "level": "INFO", "name": "httpx", "message": "HTTP Request: POST http://localhost:8000/messages/?session_id=be72cc977bdd485d98de699b36914f46 \"HTTP/1.1 202 Accepted\""}
{"timestamp": "2026-02-07T16:24:58.971808", "level": "ERROR", "name": "mcp.client.sse", "message": "Error in sse_reader", "exc_info": "Traceback (mo

In [2]:
from agent_framework.model_clients.openai.openai_client import OpenAIClient
from agent_framework.messages.client_messages import UserMessage
from agent_framework.configs.settings import settings
from agent_framework.observability.telemetry import configure_opentelemetry

# Fix: Use HTTP endpoint for traces (4318) and metrics  
configure_opentelemetry(
    service_name="agent-framework-client-test", 
    otlp_trace_endpoint="http://localhost:4318",  # Changed from 4317 (gRPC) to 4318 (HTTP)
    otlp_metric_endpoint="http://localhost:4318"
)

model_client = OpenAIClient(model="gpt-4o-mini", api_key=settings.OPENAI_API_KEY)
# response = await model_client.generate([UserMessage(content=["What is the capital of France?"])], tools=mcp_tools)
# pprint(response)

{"timestamp": "2026-02-07T16:13:06.670583", "level": "INFO", "name": "agent_framework", "message": "Configuring OpenTelemetry"}
{"timestamp": "2026-02-07T16:13:06.672295", "level": "INFO", "name": "agent_framework", "message": "Using OTLP HTTP trace exporter → http://localhost:4318/v1/traces"}


In [3]:
from agent_framework.agents.react_agent import ReActAgent

agent = ReActAgent(
    name="Test ReAct Agent",
    description="A test agent for ReAct framework",
    model_client=model_client, 
    tools=mcp_tools,
    system_instructions="""You are a helpful AI assistant. Answer the user's questions using the available tools.

CRITICAL RULES FOR TOOL USAGE:
1. If a tool call returns incomplete data, try DIFFERENT parameters:
   - For GitHub profiles: use '?tab=repositories' to see repos
   - For web pages: adjust start_index or increase max_chars
   - For searches: try different URLs or query variations

2. NEVER call the same tool with the exact same arguments more than ONCE
   - If it didn't work the first time, change your approach
   - Analyze what you got and adjust parameters accordingly

3. If you can't find the information after 2-3 different attempts, explain what you tried and suggest alternatives

4. When you have the complete answer, respond directly to the user with the information.""",
)


In [4]:
result = await agent.run("Can you search for Ravikumarchavva github profile and find top 3 repositories?")
print(result)          # clean __str__
print()
print(result.summary())  # one-liner

{"timestamp": "2026-02-07T14:23:51.305770", "level": "INFO", "name": "agent_framework", "message": "[Test ReAct Agent] Starting run: Can you search for Ravikumarchavva github profile and find top 3 repositories?..."}
{"timestamp": "2026-02-07T14:23:52.708376", "level": "INFO", "name": "httpx", "message": "HTTP Request: POST https://api.openai.com/v1/responses \"HTTP/1.1 200 OK\""}
{"timestamp": "2026-02-07T14:23:52.798935", "level": "INFO", "name": "agent_framework", "message": "[Test ReAct Agent] Step 1: tool calls → ['web_search_tool']"}
{"timestamp": "2026-02-07T14:23:52.799877", "level": "INFO", "name": "agent_framework", "message": "[Test ReAct Agent] Executing web_search_tool({'url': 'https://github.com/ravikumarchavva', 'user_agent': 'Mozilla/5.0', 'max_chars': 1000, 'start_index': 0})"}
{"timestamp": "2026-02-07T14:23:56.123594", "level": "INFO", "name": "httpx", "message": "HTTP Request: POST https://api.openai.com/v1/responses \"HTTP/1.1 200 OK\""}
{"timestamp": "2026-02-07T1

In [5]:
pprint(result)

AgentRunResult(
│   run_id='5d984477-41c8-4b45-a51e-590a3d0d2c0a',
│   agent_name='Test ReAct Agent',
│   output=[
│   │   "Here are the top 3 repositories from Ravikumarchavva's GitHub profile:\n\n1. **[agent-framework](https://github.com/Ravikumarchavva/agent-framework)**\n   - Description: Self agent framework\n   - Language: Jupyter Notebook\n   - Last Updated: Feb 7, 2026\n\n2. **[agentic_ai](https://github.com/Ravikumarchavva/agentic_ai)**\n   - Description: Custom chatbots\n   - Language: Python\n   - Last Updated: Jan 19, 2026\n\n3. **[csrio-image2biomass](https://github.com/Ravikumarchavva/csrio-image2biomass)**\n   - Description: Predicting the biomass content useful for grazing\n   - Language: Jupyter Notebook\n   - Last Updated: Nov 1, 2025\n\nIf you need more information or have other questions, feel free to ask!"
│   ],
│   status=<RunStatus.COMPLETED: 'completed'>,
│   steps=[
│   │   StepResult(
│   │   │   step=1,
│   │   │   thought=None,
│   │   │   tool_calls=[
│   │   │   │   ToolCallRecord(
│   │   │   │   │   tool_name='web_search_tool',
│   │   │   │   │   call_id='call_XWJKdFAiu97ce1TYTFN5ewYC',
│   │   │   │   │   arguments={
│   │   │   │   │   │   'url': 'https://github.com/ravikumarchavva',
│   │   │   │   │   │   'user_agent': 'Mozilla/5.0',
│   │   │   │   │   │   'max_chars': 1000,
│   │   │   │   │   │   'start_index': 0
│   │   │   │   │   },
│   │   │   │   │   result="[![View Ravikumarchavva's full-sized avatar](https://avatars.githubusercontent.com/u/109415623?v=4)](https://avatars.githubusercontent.com/u/109415623?v=4)\n\n## C.Ravi Kumar reddy Ravikumarchavva\n\n## Block or report Ravikumarchavva\n\n## Hi there! 👋\n\nI’m **Ravikumar Chavva**, a final-year Computer Science and Engineering student specializing in **Data Science** and **Machine Learning**. I’m passionate about leveraging data and AI to solve real-world problems and creating impactful solutions.\n\n---\n\n## 👩\u200d💻 About Me\n\n* 🎓 **Final-Year Student**: Pursuing B.Tech with a focus on Data Science and Machine Learning.\n* 🤖 **AI Enthusiast**: Passionate about exploring cutting-edge technologies and their practical applications.\n* 🌟 **Driven Problem-Solver**: Dedicated to building end-to-end solutions integrating ML, cloud, and full-stack technologies.\n\n---\n\n## 🛠️ Skills\n\n### Programming Languages\n\n* **Python** | **SQL** | **TypeScript**\n\n### Frameworks & Tools\n\n* **Machine Learning & AI:** Scikit-lear",
│   │   │   │   │   is_error=False,
│   │   │   │   │   duration_ms=2139.0668230014853,
│   │   │   │   │   timestamp=datetime.datetime(2026, 2, 7, 14, 23, 54, 939047)
│   │   │   │   )
│   │   │   ],
│   │   │   usage=UsageStats(prompt_tokens=283, completion_tokens=45, total_tokens=328),
│   │   │   finish_reason='tool_calls',
│   │   │   has_tool_calls=True,
│   │   │   thought_text=None
│   │   ),
│   │   StepResult(
│   │   │   step=2,
│   │   │   thought=None,
│   │   │   tool_calls=[
│   │   │   │   ToolCallRecord(
│   │   │   │   │   tool_name='web_search_tool',
│   │   │   │   │   call_id='call_z2zfUmwlPhqSj65zKY3q8waz',
│   │   │   │   │   arguments={
│   │   │   │   │   │   'url': 'https://github.com/ravikumarchavva?tab=repositories',
│   │   │   │   │   │   'user_agent': 'Mozilla/5.0',
│   │   │   │   │   │   'max_chars': 1000,
│   │   │   │   │   │   'start_index': 0
│   │   │   │   │   },
│   │   │   │   │   result="[![View Ravikumarchavva's full-sized avatar](https://avatars.githubusercontent.com/u/109415623?v=4)](https://avatars.githubusercontent.com/u/109415623?v=4)\n\n## C.Ravi Kumar reddy Ravikumarchavva\n\n## Block or report Ravikumarchavva\n\n* ### [agent-framework](/Ravikumarchavva/agent-framework) Public\n\n  self agent frame work\n\n  Jupyter Notebook\n  Updated\n\n  Feb 7, 2026\n\n\n* ### [agentic\\_ai](/Ravikumarchavva/agentic_ai) Public\n\n  custom chatbots\n\n  Python\n  Updated\n\n  Jan 19, 2026\n\n\n* ### [csrio-image2biomass](/Ravikumarchavva/csrio-image2biomass) Public\n\n  Predict

{"timestamp": "2026-02-07T14:24:06.924915", "level": "WARNING", "name": "opentelemetry.exporter.otlp.proto.grpc.exporter", "message": "Transient error StatusCode.UNAVAILABLE encountered while exporting metrics to localhost:4318, retrying in 1.14s."}


In [6]:
result_2 = await agent.run("Can you go to cricket prediction project and summarize the README?")
pprint(result_2)          # clean __str__

{"timestamp": "2026-02-07T14:24:12.153536", "level": "INFO", "name": "agent_framework", "message": "[Test ReAct Agent] Starting run: Can you go to cricket prediction project and summarize the README?..."}
{"timestamp": "2026-02-07T14:24:13.486337", "level": "ERROR", "name": "opentelemetry.exporter.otlp.proto.grpc.exporter", "message": "Failed to export metrics to localhost:4318, error code: StatusCode.UNAVAILABLE"}
{"timestamp": "2026-02-07T14:24:13.524176", "level": "INFO", "name": "httpx", "message": "HTTP Request: POST https://api.openai.com/v1/responses \"HTTP/1.1 200 OK\""}
{"timestamp": "2026-02-07T14:24:13.527923", "level": "INFO", "name": "agent_framework", "message": "[Test ReAct Agent] Step 1: tool calls → ['web_search_tool']"}
{"timestamp": "2026-02-07T14:24:13.528779", "level": "INFO", "name": "agent_framework", "message": "[Test ReAct Agent] Executing web_search_tool({'url': 'https://github.com/Ravikumarchavva/cricket-prediction', 'user_agent': 'Mozilla/5.0', 'max_chars': 

AgentRunResult(
│   run_id='06368da9-6c75-473d-bef0-d2daf32e97a8',
│   agent_name='Test ReAct Agent',
│   output=[
│   │   '### Summary of the T20 Cricket Win Prediction Project README\n\nThe **T20 Cricket Win Prediction Project** utilizes machine learning to predict the probability of a T20 cricket team winning during a match. The aim is to enhance viewer engagement through insightful analytics based on these predictions.\n\n#### Key Sections:\n\n- **Tech Stack**:\n  - **Programming Language**: Python\n  - **Machine Learning Framework**: PyTorch\n  - **Data Processing**: Apache Spark, Apache Airflow\n  - **Storage**: Hadoop HDFS\n  - **Containerization**: Docker\n  - **Visualization**: Jupyter Notebooks, Matplotlib, Seaborn\n\n- **Directory Structure**:\n  The project is organized into several folders, including configurations, data manipulation, machine learning modeling, and utility scripts.\n\n- **Data Processing Pipeline**:\n  The pipeline comprises stages such as data collection, preprocessing, merging, and filtering, orchestrated using Apache Airflow. Each step is visualized in Airflow for better tracking.\n\n- **Modeling Approach**:\n  Details on the various modeling techniques and how they are implemented are included, although specifics were not fully captured.\n\n#### Conclusion:\nThe project is designed to serve as a comprehensive framework for predicting cricket match outcomes, making it a valuable tool for cricket analytics.\n\nIf you need more detailed information or further specifics, feel free to ask!'
│   ],
│   status=<RunStatus.COMPLETED: 'completed'>,
│   steps=[
│   │   StepResult(
│   │   │   step=1,
│   │   │   thought=None,
│   │   │   tool_calls=[
│   │   │   │   ToolCallRecord(
│   │   │   │   │   tool_name='web_search_tool',
│   │   │   │   │   call_id='call_z49ApCjpGalQg34MP1ExRVHU',
│   │   │   │   │   arguments={
│   │   │   │   │   │   'url': 'https://github.com/Ravikumarchavva/cricket-prediction',
│   │   │   │   │   │   'user_agent': 'Mozilla/5.0',
│   │   │   │   │   │   'max_chars': 1000,
│   │   │   │   │   │   'start_index': 0
│   │   │   │   │   },
│   │   │   │   │   result='## T20 Cricket Win Prediction Project\n\n[![Project Overview](/Ravikumarchavva/Cricket-Prediction/raw/main/public/architectureOverview.jpg)](/Ravikumarchavva/Cricket-Prediction/blob/main/public/architectureOverview.jpg)\n\n## Overview\n\nThe **T20 Cricket Win Prediction Project** leverages advanced machine learning techniques to predict the probability of a T20 cricket team winning at any given point in a match. By integrating this model into a cricket application, we aim to enhance viewer engagement through insightful analytics based on static predictions.\n\n## Table of Contents\n\n* [Overview](#overview)\n* [Tech Stack](#tech-stack)\n* [Directory Structure](#directory-structure)\n* [Data Processing Pipeline](#data-processing-pipeline)\n* [Modeling Approach](#modeling-approach)\n* [Results](#results)\n* [Deployment](#deployment)\n* [Getting Started](#getting-started)\n* [Future Enhancements](#future-enhancements)\n* [Conclusion](#conclusion)\n\n## Tech Stack\n\n* **Programming Language**: Python\n* **Machin',
│   │   │   │   │   is_error=False,
│   │   │   │   │   duration_ms=2058.189638002659,
│   │   │   │   │   timestamp=datetime.datetime(2026, 2, 7, 14, 24, 15, 587027)
│   │   │   │   )
│   │   │   ],
│   │   │   usage=UsageStats(prompt_tokens=1172, completion_tokens=50, total_tokens=1222),
│   │   │   finish_reason='tool_calls',
│   │   │   has_tool_calls=True,
│   │   │   thought_text=None
│   │   ),
│   │   StepResult(
│   │   │   step=2,
│   │   │   thought=None,
│   │   │   tool_calls=[
│   │   │   │   ToolCallRecord(
│   │   │   │   │   tool_name='web_search_tool',
│   │   │   │   │   call_id='call_CoFigys3PAEzqbwmbufs5Jta',
│   │   │   │   │   arguments={
│   │   │   │   │   │   'url': 'https://github.com/Ravikumarchavva/cricket-prediction',
│   │   │   │   │   │   'user_agent': 'Mozilla/5.0',
│   │   │ 

In [7]:
print(result_2.output_text)  # one-liner

### Summary of the T20 Cricket Win Prediction Project README

The **T20 Cricket Win Prediction Project** utilizes machine learning to predict the probability of a T20 cricket team winning during a match. The aim is to enhance viewer engagement through insightful analytics based on these predictions.

#### Key Sections:

- **Tech Stack**:
  - **Programming Language**: Python
  - **Machine Learning Framework**: PyTorch
  - **Data Processing**: Apache Spark, Apache Airflow
  - **Storage**: Hadoop HDFS
  - **Containerization**: Docker
  - **Visualization**: Jupyter Notebooks, Matplotlib, Seaborn

- **Directory Structure**:
  The project is organized into several folders, including configurations, data manipulation, machine learning modeling, and utility scripts.

- **Data Processing Pipeline**:
  The pipeline comprises stages such as data collection, preprocessing, merging, and filtering, orchestrated using Apache Airflow. Each step is visualized in Airflow for better tracking.

- **Modeli

In [5]:
pprint(agent.memory.get_messages())

[
│   UserMessage(
│   │   id='da8eb44f-059b-4418-8e09-103313a54963',
│   │   role='user',
│   │   content=['What is the capital of France and its population?'],
│   │   type='UserMessage',
│   │   name=None
│   ),
│   AssistantMessage(
│   │   id='9e2c4b0e-c14b-41ac-b52d-9c1921eed8c8',
│   │   role='assistant',
│   │   content=None,
│   │   type='AssistantMessage',
│   │   name=None,
│   │   reasoning=None,
│   │   tool_calls=[
│   │   │   ToolCallMessage(
│   │   │   │   id='call_82XQhhFA4nCu1EptS8DbGP2t',
│   │   │   │   role='tool_call',
│   │   │   │   content=None,
│   │   │   │   type='ToolCallMessage',
│   │   │   │   name='web_search_tool',
│   │   │   │   arguments={
│   │   │   │   │   'url': 'https://en.wikipedia.org/wiki/Paris',
│   │   │   │   │   'user_agent': 'Mozilla/5.0',
│   │   │   │   │   'max_chars': 1000,
│   │   │   │   │   'start_index': 0
│   │   │   │   }
│   │   │   ),
│   │   │   ToolCallMessage(
│   │   │   │   id='call_WgIA83OrUh3QqWDZapwHpJVS',
│   │   │   │   role='tool_call',
│   │   │   │   content=None,
│   │   │   │   type='ToolCallMessage',
│   │   │   │   name='web_search_tool',
│   │   │   │   arguments={
│   │   │   │   │   'url': 'https://www.worldpopulationreview.com/world-cities/paris-population',
│   │   │   │   │   'user_agent': 'Mozilla/5.0',
│   │   │   │   │   'max_chars': 1000,
│   │   │   │   │   'start_index': 0
│   │   │   │   }
│   │   │   )
│   │   ],
│   │   finish_reason='tool_calls',
│   │   usage=UsageStats(prompt_tokens=104, completion_tokens=107, total_tokens=211),
│   │   cached=False
│   ),
│   ToolExecutionResultMessage(
│   │   id='aeb4d6ec-5ddf-4e0e-b380-04e312627a99',
│   │   role='tool_response',
│   │   content=[
│   │   │   {
│   │   │   │   'type': 'text',
│   │   │   │   'text': "Error calling tool 'web_search_tool': Failed to fetch https://en.wikipedia.org/wiki/Paris - status code 403"
│   │   │   }
│   │   ],
│   │   type='ToolExecutionResultMessage',
│   │   tool_call_id='call_82XQhhFA4nCu1EptS8DbGP2t',
│   │   name='web_search_tool',
│   │   isError=True
│   ),
│   ToolExecutionResultMessage(
│   │   id='ff35bf3f-1360-46fa-9972-f9d2ce179428',
│   │   role='tool_response',
│   │   content=[
│   │   │   {
│   │   │   │   'type': 'text',
│   │   │   │   'text': '[Redirecting from `/world-cities/paris-population` to `/cities/france/paris`](/cities/france/paris)'
│   │   │   }
│   │   ],
│   │   type='ToolExecutionResultMessage',
│   │   tool_call_id='call_WgIA83OrUh3QqWDZapwHpJVS',
│   │   name='web_search_tool',
│   │   isError=False
│   ),
│   AssistantMessage(
│   │   id='994afa64-2e18-47d2-a707-46ef6831bc7e',
│   │   role='assistant',
│   │   content=None,
│   │   type='AssistantMessage',
│   │   name=None,
│   │   reasoning=None,
│   │   tool_calls=[
│   │   │   ToolCallMessage(
│   │   │   │   id='call_QGO7rjVKTxczfNd3mFjrIIgz',
│   │   │   │   role='tool_call',
│   │   │   │   content=None,
│   │   │   │   type='ToolCallMessage',
│   │   │   │   name='web_search_tool',
│   │   │   │   arguments={
│   │   │   │   │   'url': 'https://en.wikipedia.org/wiki/Paris',
│   │   │   │   │   'user_agent': 'Mozilla/5.0',
│   │   │   │   │   'max_chars': 1000,
│   │   │   │   │   'start_index': 0
│   │   │   │   }
│   │   │   ),
│   │   │   ToolCallMessage(
│   │   │   │   id='call_fhUPsUEUJvwNzxaJjZ1kRN1o',
│   │   │   │   role='tool_call',
│   │   │   │   content=None,
│   │   │   │   type='ToolCallMessage',
│   │   │   │   name='web_search_tool',
│   │   │   │   arguments={
│   │   │   │   │   'url': 'https://www.worldpopulationreview.com/cities/france/paris',
│   │   │   │   │   'user_agent': 'Mozilla/5.0',
│   │   │   │   │   'max_chars': 1000,
│   │   │   │   │   'start_index': 0
│   │   │   │   }
│   │   │   )
│   │   ],
│   │   finish_reason='tool_calls',
│   │   usage=UsageStats(prompt_tokens=274, completion_tokens=106, total_tokens=380),
│   │   cached=False
│   ),
│   ToolExecutionResultMessage(
│   │   id='85f507bb-9d88-46ec-b983-649de908db2d',
│   │

{"timestamp": "2026-02-07T16:14:06.686889", "level": "WARNING", "name": "opentelemetry.exporter.otlp.proto.grpc.exporter", "message": "Transient error StatusCode.UNAVAILABLE encountered while exporting metrics to localhost:4318, retrying in 1.15s."}
{"timestamp": "2026-02-07T16:14:13.006107", "level": "ERROR", "name": "opentelemetry.exporter.otlp.proto.grpc.exporter", "message": "Failed to export metrics to localhost:4318, error code: StatusCode.UNAVAILABLE"}
{"timestamp": "2026-02-07T16:15:13.008582", "level": "WARNING", "name": "opentelemetry.exporter.otlp.proto.grpc.exporter", "message": "Transient error StatusCode.UNAVAILABLE encountered while exporting metrics to localhost:4318, retrying in 1.08s."}
{"timestamp": "2026-02-07T16:15:19.789086", "level": "ERROR", "name": "opentelemetry.exporter.otlp.proto.grpc.exporter", "message": "Failed to export metrics to localhost:4318, error code: StatusCode.UNAVAILABLE"}
{"timestamp": "2026-02-07T16:16:19.791479", "level": "WARNING", "name": 

## Streaming API Test

Now testing the new streaming implementation with typed events:
- **TextDeltaChunk**: Incremental text content as it's generated
- **ReasoningDeltaChunk**: Thinking/reasoning from o1/o3 models
- **ToolCallChunk**: Complete tool call information
- **CompletionChunk**: Final response with full AssistantMessage

The agent will:
1. Yield chunks as they arrive from the LLM
2. Add the final complete message to memory
3. Continue with tool execution if needed

In [4]:
# Test streaming with typed events
from agent_framework.messages import TextDeltaChunk, ReasoningDeltaChunk, CompletionChunk

agent.reset()

print("🌊 Starting streaming run...\n")
async for chunk in agent.run_stream("What is the capital of France and its population?"):
    if isinstance(chunk, TextDeltaChunk):
        print(chunk.text, end="", flush=True)
    elif isinstance(chunk, ReasoningDeltaChunk):
        print(f"\n🧠 Reasoning: {chunk.text}", end="", flush=True)
    elif isinstance(chunk, CompletionChunk):
        print(f"\n\n✅ Completion: {chunk.message.finish_reason}")
        if chunk.message.usage:
            print(f"   Usage: {chunk.message.usage}")

print("\n\n📚 Memory after streaming:")
pprint(agent.memory.get_messages()[-3:])

🌊 Starting streaming run...

{"timestamp": "2026-02-07T16:13:11.502749", "level": "INFO", "name": "agent_framework", "message": "[Test ReAct Agent] Starting streaming run: What is the capital of France and its population?..."}
{"timestamp": "2026-02-07T16:13:12.456438", "level": "INFO", "name": "httpx", "message": "HTTP Request: POST https://api.openai.com/v1/responses \"HTTP/1.1 200 OK\""}


✅ Completion: tool_calls
   Usage: prompt_tokens=104 completion_tokens=107 total_tokens=211
{"timestamp": "2026-02-07T16:13:13.996674", "level": "INFO", "name": "agent_framework", "message": "[Test ReAct Agent] [stream] Step 1: tools → ['web_search_tool', 'web_search_tool']"}
{"timestamp": "2026-02-07T16:13:13.997728", "level": "INFO", "name": "agent_framework", "message": "[Test ReAct Agent] Executing web_search_tool({'url': 'https://en.wikipedia.org/wiki/Paris', 'user_agent': 'Mozilla/5.0', 'max_chars': 1000, 'start_index': 0})"}
{"timestamp": "2026-02-07T16:13:14.227895", "level": "INFO", "name

[
│   ToolExecutionResultMessage(
│   │   id='85f507bb-9d88-46ec-b983-649de908db2d',
│   │   role='tool_response',
│   │   content=[
│   │   │   {
│   │   │   │   'type': 'text',
│   │   │   │   'text': "Error calling tool 'web_search_tool': Failed to fetch https://en.wikipedia.org/wiki/Paris - status code 403"
│   │   │   }
│   │   ],
│   │   type='ToolExecutionResultMessage',
│   │   tool_call_id='call_QGO7rjVKTxczfNd3mFjrIIgz',
│   │   name='web_search_tool',
│   │   isError=True
│   ),
│   ToolExecutionResultMessage(
│   │   id='62c10af9-a4a9-4c1e-9499-439f042151de',
│   │   role='tool_response',
│   │   content=[
│   │   │   {
│   │   │   │   'type': 'text',
│   │   │   │   'text': '11.4M\n\nTotal Population\n\n71.5K\n0.63%\n\nGrowth\n\nYes\n\nCapital of France?\n\n## Paris Overview\n\nParis is a bustling city that is full of life and flavor. As the capital city of France, the city is home to numerous government buildings, but it is known for its quaint cobblestone paved streets and rich historical landmarks.\n\n## Landmark of Paris - Eiffel Tower\n\nOne of the most infamous landmarks in Paris is the Eiffel Tower. Each year, the Eiffel Tower has [almost 7 million visitors,](http://www.toureiffel.paris/en/everything-about-the-tower/the-eiffel-tower-at-a-glance.html) and in 2010, the Eiffel Tower had its 250 millionth visitor.\n\n## Paris Population Size\n\nThe rest of the city is often just as bustling with activity of city life, with a 2016 [estimated population](https://en.wikipedia.org/wiki/Demographics_of_Paris) of 2.2 million residents living within its city limits.\n\n## Population of Paris Demographics\n\nAccording to recent estimates, the population of the city of Paris is [2,206,488'
│   │   │   }
│   │   ],
│   │   type='ToolExecutionResultMessage',
│   │   tool_call_id='call_fhUPsUEUJvwNzxaJjZ1kRN1o',
│   │   name='web_search_tool',
│   │   isError=False
│   ),
│   AssistantMessage(
│   │   id='86ea2769-fc8a-4a99-898e-25cf53ba2196',
│   │   role='assistant',
│   │   content=[
│   │   │   'The capital of France is **Paris**. As of recent estimates, its population is approximately **2.2 million** residents within the city limits. The larger metropolitan area has a population of around **11.4 million**.'
│   │   ],
│   │   type='AssistantMessage',
│   │   name=None,
│   │   reasoning=None,
│   │   tool_calls=None,
│   │   finish_reason='stop',
│   │   usage=UsageStats(prompt_tokens=652, completion_tokens=48, total_tokens=700),
│   │   cached=False
│   )
]